import numpy as np
import pandas as pd

In [3]:
header = ['user_id','item_id','rating','timestamp']
df = pd.read_csv('/Users/psrini1/Documents/Python/Recommendation system/ml-100k/u.data', sep ='\t', names = header)
## Extracting specific columns from the dataframe
select = ["user_id","item_id"]
df[select]

## Getting the unique count of users / items 

users = df.user_id.unique().shape[0]
users ## 943unique users
items = df.item_id.unique().shape[0]
items ## 1682 unique movies

## Splitting the data using CV 
from sklearn import cross_validation as cv
train_data,test_data = cv.train_test_split(df,test_size = 0.25)

## Creating the user-item matrix 
train_data_matrix = np.zeros((users,items))
train_data_matrix

test_data_matrix = np.zeros((users,items))

## Check what's stored in line
##for line in train_data.itertuples():
  ##  print(line)

for line in train_data.itertuples():
    train_data_matrix [line[1]-1,line[2]-1]= line[3]

for line in test_data.itertuples():
    test_data_matrix[line[1]-1,line[2]-1] =line[3]

/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
## Calculating item and user based similarity scores 

from sklearn.metrics.pairwise import pairwise_distances  

user_similarity = pairwise_distances(train_data_matrix,metric ='cosine')

item_similarity = pairwise_distances(train_data_matrix.T,metric = 'cosine')

In [5]:
a = np.abs(item_similarity).sum(axis=1)
a

array([ 1454.2260641 ,  1479.78585468,  1525.07625219, ...,  1669.43636309,
        1652.27459153,  1655.3895763 ])

In [6]:
a = np.ndarray(shape=(2,2),dtype = int, order = 'F')
b = np.abs(a)
b

array([[0, 2],
       [1, 3]])

In [7]:
## User-user CF

## Averge rating for each user
def prediction(rating,similarity,type):
    if type == 'user':
        mean_rating = rating.mean(axis=1)
        rating_diff = rating-mean_rating[:,np.newaxis]
        pred = mean_rating[:,np.newaxis] + similarity.dot(rating_diff)/ np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = rating.dot(similarity)/np.array([np.abs(similarity).sum(axis=1)])
    return pred


In [9]:
item_prediction = prediction(train_data_matrix,item_similarity,type = 'item')
user_prediction = prediction(train_data_matrix,user_similarity,type ='user')

In [10]:
user_prediction.shape

(943, 1682)

In [13]:
c1 = test_data["user_id"] == 1
c2 = test_data["item_id"] == 1

##train_data[train_data["user_id"] == 38 & train_data["item_id"] == 758]
test_data[c1 & c2]
test_data_matrix[0,0]

5.0

In [47]:
## Evaluation of the model predictions 

from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse_eval(actual,prediction):
    prediction = prediction[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(actual,prediction))

rmse_eval(test_data_matrix,user_prediction)

3.1253595439463076

In [31]:
def rmse_eval1(actual,prediction):
    actual = actual[actual.nonzero()].flatten()
    prediction = prediction[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction,actual))

rmse_eval1(test_data_matrix,user_prediction)

IndexError: index 943 is out of bounds for axis 0 with size 943

In [42]:
test_data_matrix[test_data_matrix.nonzero()].flatten()

array([ 5.,  3.,  5., ...,  5.,  3.,  3.])

In [45]:
user_prediction[test_data_matrix.nonzero()].flatten()

array([ 1.6352657 ,  0.56878181,  0.37196922, ...,  0.29504158,
        0.17447012,  0.16571898])